In [18]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

import pygame
from pygame import mixer

frescos = {
    # "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    # "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    # "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    # "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    # "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    # "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    # "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

#Play sound when completed
mixer.init()
mixer.music.load(".././assets/completed.mp3")
mixer.music.set_volume(0.4)
mixer.music.play()  

df_alcampo



Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=0
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=0
Category: 0 Name: 0 Image: 0 Price: 0 Stock 0
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=1
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=2
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=3
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=4
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=5
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Quesos https://www.alcampo.

,Category,Name,Price,Stock,Image
0,Quesos,Queso en porciones PRODUCTO ALCAMPO 16 uds.250 g.,"1,91 €(7,64 €/Kg)",Yes,https://www.alcampo.es/media/hee/hce/981124166...
1,Quesos,Queso feta ALCAMPO GOURMET 150 g.,"2,64 €(17,60 €/Kg)",Yes,https://www.alcampo.es/media/h22/h4d/966486769...
2,Quesos,"Queso rallado mezcla 4 quesos, para fundir y g...","2,10 €(14,00 €/Kg)",Yes,https://www.alcampo.es/media/hf3/h7f/109775138...
3,Quesos,"Queso semicurado de vaca, cabra y oveja GARCíA...","(15,92 €/Kg)",Yes,https://www.alcampo.es/media/h34/h13/889229338...
4,Quesos,"Queso grana padano en cuña PRODUCTO ALCAMPO, 2...","2,94 €(14,70 €/Kg)",Yes,https://www.alcampo.es/media/h9e/h2a/116541322...
...,...,...,...,...,...
1027,Fresco al Corte,"Medio cordero lechal, especial para asados y c...","69,83 €(19,95 €/Kg)",Yes,https://www.alcampo.es/media/h33/h0e/109069259...
1028,Fresco al Corte,Chuletón de vacuno con IGP carne de la Sierra ...,"16,76 €(20,95 €/Kg)",No,https://www.alcampo.es/media/hae/ha2/109310886...
1029,Fresco al Corte,Chicharro mostrador,"5€(9,99 €/Kg)",Yes,https://www.alcampo.es/media/h55/hb1/110263468...
1030,Fresco al Corte,"Solomillo de vaca madurada, especial para plan...","10,49 €(34,95 €/Kg)",No,https://www.alcampo.es/media/h18/h71/109087437...


In [19]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

# read_file = pd.read_csv ("../data/alcampo.csv")
# read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

alcampo = pd.read_csv("../data/alcampo.csv")
print("File saved",today)
alcampo

File saved 2023-03-18 14:15:24.611694


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
207007,Fresco al Corte,"Medio cordero lechal, especial para asados y c...","69,83 €(19,95 €/Kg)",Yes,https://www.alcampo.es/media/h33/h0e/109069259...,Alcampo,2023-03-18 14:15:24.611694
207008,Fresco al Corte,Chuletón de vacuno con IGP carne de la Sierra ...,"16,76 €(20,95 €/Kg)",No,https://www.alcampo.es/media/hae/ha2/109310886...,Alcampo,2023-03-18 14:15:24.611694
207009,Fresco al Corte,Chicharro mostrador,"5€(9,99 €/Kg)",Yes,https://www.alcampo.es/media/h55/hb1/110263468...,Alcampo,2023-03-18 14:15:24.611694
207010,Fresco al Corte,"Solomillo de vaca madurada, especial para plan...","10,49 €(34,95 €/Kg)",No,https://www.alcampo.es/media/h18/h71/109087437...,Alcampo,2023-03-18 14:15:24.611694
